In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode  # noqa: F401
except ImportError:
    !python -m pip install openmdao[notebooks]

<script type="text/x-mathjax-config">
  MathJax.Hub.Config({
    TeX: {
      equationNumbers: {
        autoNumber: "AMS",
        useLabelIds: true
      }
    },
    displayAlign: "center"
  });
</script>

# Post-Optimality Sensitivities

Post-optimality sensitivities refer to the derivative of an optimization output wrt an optimization input.
These sensitivities are intended to inform the user of the cost of a design choice.
**These sensitivities only apply in the context of gradient-based optimization.**

## Viewing optimization as a nonlinear solve

From a MAUD perspective, one could view a gradient-based optimization as a nonlinear solve.

The implicit outputs in this context are the design varaible values, and the residual
equations come from the Karush-Kuhn-Tucker conditions for optimization.

### The Unconstrained Case

Consider the optimization of some function. Here is the standard form of the OpenMDAO 
paraboloid where we've replaced the constants with some parameters $p_0$, $p_1$, and $p_2$.

\begin{align}
\min_{x,\, y} \quad 
    & f(x, y; \mathbf{p}) = (x - p_0)^2 + x y + (y + p_1)^2 - p_2 \\
    \text{where} \quad 
    & \mathbf{p} = \begin{bmatrix} 3 \\ 4 \\ 3 \end{bmatrix} \in \mathbb{R}^3 \\
    & x, y \in \mathbb{R}
\end{align}

This system has 3 outputs ($f$, $x$, $y$) and 3 inputs ($\bar{p}$). So we may be interested in
in how those outputs change at the next converged solution subject to small changes in $\bar{p}$.

\begin{align}
    \frac{d x^*}{d \bar{p}} \\
    \frac{d y^*}{d \bar{p}} \\
    \frac{d f^*}{d \bar{p}} \\
\end{align}

In an unconstrained problem, the optimizer is finding the "bottom" of a multidimensional bowl:

\begin{align*}
  \mathcal{R_{KKT}}(\bar\theta, \bar{p}) = \nabla_\theta f(\bar{\theta}, \bar{p}) &= \bar{0}
\end{align*}

Where $\theta$ here represents the design variables instead of the typical $x$ in order to not confuse it with
the inputs the to the paraboloid above.
With two design variables, the residual vector $\mathcal{R_{KKT}}$ has two elements.

Our residual system has the following structure.

\begin{align*}
  \begin{bmatrix}
      \frac{df^*}{d\bar{p}} \\
  \end{bmatrix}
  &=
  \begin{bmatrix}
      \frac{df}{d\bar{p}} \\
  \end{bmatrix}
  -
  \begin{bmatrix}
      \frac{df}{d\bar{\theta}} \\
  \end{bmatrix}
  \cancel{
    \begin{bmatrix}
        \frac{d \nabla_{\theta} f}{d\bar{\theta}}  \\
    \end{bmatrix}^{-1}
  }
  \begin{bmatrix}
      \frac{d \nabla_{\theta} f}{d\bar{p}} \\
  \end{bmatrix}
\end{align*}

or, expanding terms...

\begin{align*}
  \begin{bmatrix}
      \frac{df^*}{dp_0} & \frac{df^*}{dp_1} & \frac{df^*}{dp_2} \\
  \end{bmatrix}
  &=
  \begin{bmatrix}
      \frac{df}{dp_0} & \frac{df}{dp_1} & \frac{df}{dp_2} \\
  \end{bmatrix}
  -
  \begin{bmatrix}
      \frac{df}{dx} & \frac{df}{dy} \\
  \end{bmatrix}
  \cancel{
    \begin{bmatrix}
        \frac{d\nabla_x f}{dx} & \frac{d\nabla_x f}{dy} \\
        \frac{d\nabla_y f}{dx} & \frac{d\nabla_y f}{dy} \\
    \end{bmatrix}^{-1}
  }
  \begin{bmatrix}
      \frac{d\nabla_x f}{dp_0} & \frac{d\nabla_x f}{dp_1} & \frac{d\nabla_x f}{dp_2} \\
      \frac{d\nabla_y f}{dp_0} & \frac{d\nabla_y f}{dp_1} & \frac{d\nabla_y f}{dp_2} \\
  \end{bmatrix}
\end{align*}

In an unconstrained optimization such as this, the Hessian $\left(\frac{d\nabla_{\theta} f}{d\theta}\right)$ is zero, so the post-optimality sensitivity of the objective wrt the parameters is identical to the total derivative of the objective wrt the parameters.

The following code generates plots that sweep the value of each parameter and plot the resulting objective value, overlaid by vectors indicating the calculated derivative $\frac{df^*}{dp_i}$.

In [ ]:
import numpy as np
import openmdao.api as om
import matplotlib.pyplot as plt

prob = om.Problem()
prob.model.add_subsystem('parab', om.ExecComp('f_xy = (x-p0)**2 + x*y + (y+p1)**2 - p2'),
                            promotes_inputs=['x', 'y', 'p0', 'p1', 'p2'],
                            promotes_outputs=['f_xy'])

# Design variables 'x' and 'y' span components, so we need to provide a common initial
# value for them.
prob.model.set_input_defaults('x', 3.0)
prob.model.set_input_defaults('y', -4.0)
prob.model.set_input_defaults('p0', 3.0)
prob.model.set_input_defaults('p1', 4.0)
prob.model.set_input_defaults('p2', 3.0)

# setup the optimization
prob.driver = om.pyOptSparseDriver()
prob.driver.options['print_results'] = False
prob.driver.options['optimizer'] = 'SLSQP'
prob.driver.options['singular_jac_behavior'] = 'ignore'

prob.model.add_design_var('x', lower=-50, upper=50)
prob.model.add_design_var('y', lower=-50, upper=50)
prob.model.add_objective('f_xy')

prob.setup()

prob.run_model()
driver_vars = prob.list_driver_vars(out_stream=None)
des_vars = [dv for dv, _ in driver_vars['design_vars']]
constraints = [dv for dv, _ in driver_vars['constraints']]
objs = [dv for dv, _ in driver_vars['objectives']]
other_ofs = []
other_wrts = ['p0', 'p1', 'p2']

ofs = objs + constraints + other_ofs
wrts = des_vars + other_wrts

fig, axs = plt.subplots(3, 1)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=None)
ax1 = axs[0]
ax2 = axs[1]
ax3 = axs[2]

n_pts = 10
ps = np.linspace(0, 10, n_pts)
f_stars = np.zeros(n_pts)
df_stars_dps = np.zeros((3, n_pts))

prob.run_driver()
# No constraints, no lagrange multipliers
totals = prob.compute_totals(of=ofs, wrt=wrts, driver_scaling=False)

print('Nominal Solution:')
print('   f*:', prob.get_val('f_xy'))
print('   x*:', prob.get_val('x'))
print('   y*:', prob.get_val('y'))
print('   df*/dp0:', totals['f_xy', 'p0'])
print('   df*/dp1:', totals['f_xy', 'p1'])
print('   df*/dp2:', totals['f_xy', 'p2'])


for j in range(3):

    prob.set_val('p0', 3)
    prob.set_val('p1', 4)
    prob.set_val('p2', 3)

    for i, p in enumerate(ps):
        prob[f'p{j}'] = p
        prob.run_driver()
        f_stars[i] = np.copy(prob['f_xy'])[0]

        est_multipliers, _ = prob.driver._get_lagrange_multipliers(driver_scaling=False, feas_tol=1.0E-6)

        totals = prob.compute_totals(of=ofs, wrt=wrts, driver_scaling=False)

        # dfstar_dg = est_multipliers['g']
        # dg_dp = totals['g', 'p0']
        dfstar_dpj= totals['f_xy', f'p{j}']

        # df_stars_dps[i] = np.copy(est_multipliers['g'] * (prob['x'] + prob['y']))
        df_stars_dps[j, i] = np.copy(dfstar_dpj).ravel()[0]



    uv = np.column_stack([np.ones_like(df_stars_dps[j]),
                            df_stars_dps[j]])

    uv_norm = np.linalg.norm(uv, axis=1)
    uv /= uv_norm[:, np.newaxis]

    axs[j].plot(ps,
                f_stars,
                zorder=0,
                linewidth=2.0)
    axs[j].quiver(ps,
                f_stars,
                uv[:,0],
                uv[:,1],
                angles="xy",
                pivot="mid",
                color="red",
                width=0.01,
                headwidth=2,
                headlength=1.0,
                headaxislength=1.0,
                minshaft=0.1,
                minlength=0.5)

    axs[j].set(xlabel=f'$p_{j}$', ylabel=r'$f^*$')
    axs[j].grid()

fig.tight_layout()

plt.show()

## Optimal design variable sensitivities for unconstrained optimization

In the section above, we demonstrate the ability to get the post-optimality sensitivity of the objective with respect to some parameters of the problem.

Now we want to know, how would the optimal values of $x$ and $y$ change as the values of $\bar{p}$ are changed?
The KKT residual is still the same for the unconstrained case.
Viewing the problem as an implicit solve, we have the following linear system for the residuals

\begin{align*}
  \begin{bmatrix}
      \frac{d{\theta}^*}{d\bar{p}} \\
  \end{bmatrix}
  &=
  \cancel{
    \begin{bmatrix}
      \frac{d{\theta}}{d{\bar{p}}} \\
    \end{bmatrix}
  }
  -
  \begin{bmatrix}
      \frac{d{\theta}}{d{\theta}} \\
  \end{bmatrix}
  \begin{bmatrix}
    \frac{d\nabla_{\theta} {f}}{d{\theta}} \\
  \end{bmatrix}^{-1}
  \begin{bmatrix}
      \frac{d\nabla_{\theta} {f}}{d\bar{p}} \\
  \end{bmatrix}
\end{align*}

The term $\frac{d{\theta}}{d{\bar{p}}}$ drops to zero because the unoptimized design variables are not impacted by the choice of the parameters.
The term $\frac{d{\theta}}{d{\theta}}$ is just an identity matrix, so we're left with:

\begin{align*}
  \begin{bmatrix}
      \frac{d{\theta}^*}{d\bar{p}} \\
  \end{bmatrix}
  &=
  -
  \begin{bmatrix}
    \frac{d\nabla_{\theta} {f}}{d{\theta}} \\
  \end{bmatrix}^{-1}
  \begin{bmatrix}
      \frac{d\nabla_{\theta} {f}}{d\bar{p}} \\
  \end{bmatrix}
\end{align*}

or equivalently

\begin{align*}
  \begin{bmatrix}
    \frac{d\nabla_{\theta} {f}}{d{\theta}} \\
  \end{bmatrix}
  \begin{bmatrix}
      \frac{d{\theta}^*}{d\bar{p}} \\
  \end{bmatrix}
  &=
  -
  \begin{bmatrix}
      \frac{d\nabla_{\theta} {f}}{d\bar{p}} \\
  \end{bmatrix}
\end{align*}

So given $\frac{d\nabla_{\theta} {f}}{d{\theta}}$ and $\frac{d\nabla_{\theta} {f}}{d\bar{p}}$, a linear solve provides the sensitivities $\frac{d{\theta}^*}{d\bar{p}}$.

But how do we obtain these?

A derivative of a gradient is a second derivative, and OpenMDAO currently does not provide second derivatives.

### Enter the Lagrange Multipliers

In a constrained optimization problem, Lagrange multiplier terms are included in the KKT residual for each active constraint.
The presence of active constraints means that the optimization will not find the "bottom of the bowl", but rather be limited to finding some minimal value on the surface of the bowl where the active constraint residual is zero.

\begin{align*}
  \mathcal{R_{KKT}} =
  \begin{bmatrix}
  \nabla_x f + \nabla_x \bar{g}^T \bar\lambda + \nabla_x \bar{h}^T_\mathcal{A} \bar\mu_\mathcal{A} + \bar\nu_\mathcal{A}
  \end{bmatrix} &= \bar{0}
\end{align*}

Where $\bar{g}$ are the equality constraints, $\bar{h}_\mathcal{A}$ are the active inequality constraints, and $\bar\nu_\mathcal{A}$

Lumping all active constraints and bounds into a single pseudo-equality-constraint vector $\mathcal{G}$, we have

\begin{align*}
  \nabla_x \mathcal{G}^T \begin{bmatrix}\bar\lambda \\ \bar\mu \\ \bar\nu \end{bmatrix} = -\nabla_x f
\end{align*}

The gradients in the above equation are obtained by a call to OpenMDAO's `problem.compute_totals` method.

From this we can find the multipliers using a least squares method in order to allow a little slack due to numerical issues.
These multipliers give the _post-optimality sensitivities_ of the objective $f$ with respect to the active constraint bound values for the equality constraints, inequality constraints, and design variables, respectively.

Now suppose, having already found the optimal solution that minimizes $f$, we treat the design variable vector $\theta$ as if it were equality-constrained to those optimal values:

\begin{align*}
  \mathcal{G} = \begin{bmatrix}x - x* \\ y - y* \end{bmatrix} &= \left[ 0 \right]
\end{align*}

If we were to perturb the design variables $x$ and $y$ away from their optimal values and solve the KKT system, the resulting multpliers would approximate the sensitivities of the objective with respect to the design variables, $\frac{d\nabla_{\theta} {f}}{d{\theta}}$.

Similarly, a perturbation of the parameter values and treating them as if they were constraints would give us $\frac{d\nabla_{\theta} {f}}{d{\bar{p}}}$.


In [ ]:
import numpy as np
import openmdao.api as om
import matplotlib.pyplot as plt

prob = om.Problem()
prob.model.add_subsystem('parab', om.ExecComp('f_xy = (x-p0)**2 + x*y + (y+p1)**2 - p2'),
                            promotes_inputs=['x', 'y', 'p0', 'p1', 'p2'],
                            promotes_outputs=['f_xy'])

# Design variables 'x' and 'y' span components, so we need to provide a common initial
# value for them.
prob.model.set_input_defaults('x', 3.0)
prob.model.set_input_defaults('y', -4.0)
prob.model.set_input_defaults('p0', 3.0)
prob.model.set_input_defaults('p1', 4.0)
prob.model.set_input_defaults('p2', 3.0)

# setup the optimization
prob.driver = om.pyOptSparseDriver()
prob.driver.options['print_results'] = False
prob.driver.options['optimizer'] = 'SLSQP'
prob.driver.options['singular_jac_behavior'] = 'ignore'

prob.model.add_design_var('x', lower=-50, upper=50)
prob.model.add_design_var('y', lower=-50, upper=50)
prob.model.add_objective('f_xy')

prob.setup()

PERTURB_H = 1.0E-6

p0 = 3.
p1 = 4.
p2 = 3.
prob.set_val('p0', p0)
prob.set_val('p1', p1)
prob.set_val('p2', p2)

# prob[f'p{j}'] = p
prob.run_driver()
# x_stars[i] = np.copy(prob['x'])[0]
# y_stars[i] = np.copy(prob['y'])[0]

fstar = prob.get_val('f_xy')
xstar = prob.get_val('x')
ystar = prob.get_val('y')

print('f*', fstar)
print('x*', xstar)
print('y*', ystar)

print()

# The nominal multipliers
nom_mult, _ = prob.driver._get_lagrange_multipliers(driver_scaling=False, feas_tol=1.0E-6)

driver_vars = prob.list_driver_vars(out_stream=None)
des_vars = [dv for dv, _ in driver_vars['design_vars']]
constraints = [dv for dv, _ in driver_vars['constraints']]
objs = [dv for dv, _ in driver_vars['objectives']]
other_ofs = []
other_wrts = ['p0', 'p1', 'p2']

ofs = objs + constraints + other_ofs
wrts = des_vars + other_wrts

totals = prob.compute_totals(of=ofs, wrt=wrts, driver_scaling=False)

# Perturb the design variables to obtain dgradf/dtheta
# First do x
prob.set_val('x', xstar + PERTURB_H)
prob.run_model()

# The perturbed multipliers
pert_mult, _ = prob.driver._get_lagrange_multipliers(driver_scaling=False, feas_tol=1.0E-6, assume_dvs_active=True)
prob.set_val('x', xstar)
dgradf_dx = pert_mult['x'] / PERTURB_H
dgradf_dy = pert_mult['y'] / PERTURB_H

print('\nperturbed x multipliers')
print(f'{dgradf_dx[0]=}')
print(f'{dgradf_dy[0]=}')

# Now do y
prob.set_val('y', ystar + PERTURB_H)
prob.run_model()

# The perturbed y multipliers
pert_mult, _ = prob.driver._get_lagrange_multipliers(driver_scaling=False, feas_tol=1.0E-6, assume_dvs_active=True)
prob.set_val('y', ystar)
dfstar_dx = pert_mult['x'] / PERTURB_H
dfstar_dy = pert_mult['y'] / PERTURB_H

print('\nperturbed y multipliers')
print(f'{dfstar_dx[0]=}')
print(f'{dfstar_dy[0]=}')

# Perturb the parameters to obtain dgradf/dp
for p_idx in range(3):

    dfstar_dpi = totals['f_xy', f'p{p_idx}']

    # print('nominal multipliers')
    # print(nom_mult)
    # prob[f'p{j}'] = p + PERTURB_H
    p_save = prob.get_val(f'p{p_idx}')
    prob.set_val(f'p{p_idx}', p0 + PERTURB_H)
    prob.run_model()

    # The perturbed multipliers
    pert_mult, _ = prob.driver._get_lagrange_multipliers(driver_scaling=False, feas_tol=1.0E-6, assume_dvs_active=True)
    prob.set_val(f'p{p_idx}', p_save)

    # print('nominal totals')
    # print(totals)

    print('\nperturbed multipliers')
    # print(pert_mult)
    # print([(name, pm_val) for name, pm_val in pert_mult.items()])
    dfstar_dx = pert_mult['x'] / PERTURB_H
    dfstar_dy = pert_mult['y'] / PERTURB_H
    print('df*/dx', dfstar_dx)
    print('df*/dy', dfstar_dy)

    # The perturbed multiplier on x is df*/dx*

    # The totals of f wrt p0 is
    print(f'df*/dp{p_idx}', dfstar_dpi)


These equations are used to solve for the design variables and various lagrange multipliers.  Due to numerical peculiarities in the optimizer, solving this system for both the design variables and the multipliers ($\lambda$, $\mu$, $\nu$) may yield different results than those obtained by the optimizer.

Instead, use the same value of the design variables ($\bar{x}$) that was obtained from the optimization.
Then this residual is just a function of the multipliers, and we can remove the equality constraints from the residual, since they are not a function of the multipliers.

The corresponding residual vector is now

\begin{align*}
  \mathcal{R_{KKT}} =
  \begin{bmatrix}
  \nabla_x f + \nabla_x \bar{g}^T \bar\lambda + \nabla_x \bar{h}^T(\bar\mu_{hl} - \bar\mu_{hu}) + \bar\nu_{xl} - \bar\nu_{xu} &(5)\\
  \mu_{hl} \odot \left[ \bar{h}(\bar{x}, \bar{p}) - h_l \right]  &(2)\\
  \mu_{hu} \odot \left[ h_u - \bar{h}(\bar{x}, \bar{p}) \right]  &(2)\\
  \nu_l \odot (x - x_l)  &(5)\\ 
  \nu_u \odot (x_u - x)  &(5)\\ 
  \end{bmatrix} &= \bar{0}
\end{align*}

Note that this vector size is 19, but the number of lagrange multipliers is 16 (2 $\lambda$, 4 $\mu$, 10 $\nu$). So we can't necessarily solve this system at the $x$ found by the optimizer.

## Inactive constraints

Inactive constraints do not contribute to the problem (no sensitivities, residuals zero by default).

Instead, we parse out the active constraints and bounds, and require that the residuals associated with those be exactly zero. From this perspective the KKT residual is just:

\begin{align*}
  \mathcal{R_{KKT}} =
  \begin{bmatrix}
  \nabla_x f + \nabla_x \bar{g}^T \bar\lambda + \nabla_x \bar{h}^T_\mathcal{A} \bar\mu_\mathcal{A} + \bar\nu_\mathcal{A}
  \end{bmatrix} &= \bar{0}
\end{align*}

Lumping all active constraints and bounds into a single pseudo-constraint vector $\mathcal{G}$, we have

\begin{align*}
  \nabla_x \mathcal{G}^T \begin{bmatrix}\bar\lambda \\ \bar\mu \\ \bar\nu \end{bmatrix} = -\nabla_x f
\end{align*}

For which we can find the multipliers using a least squares method in order to allow a little slack due to numerical issues.

